# Capella API: Search, Order, and Download Tutorial

In [ ]:
# Required libraries:
# requests
# json
# folium
# urllib

In [ ]:
!pip install folium # skip this if library is already installed

Your username and password must be saved in a .json file named 'credentials.json' and formatted as follows.

{"username": "yourusername","password": "xxxxxxxxx"}

### Import required libraries, build a print utility function, assign API endpoints and load Credentials

In [ ]:
import requests
import json

# JSON utility function
def p(data):
    print(json.dumps(data, indent=2))

# Capella API endpoints
URL = 'https://api.capellaspace.com'
token = '/token'
collections = '/catalog/collections'
catsearch = '/catalog/search'
orders = '/orders/'

#Load username and password
with open('credentials.json') as f:
    data = json.load(f)
    username = data['username']
    password = data['password']

### Get and Print Access Token

In [ ]:
#Get the token
r = requests.post(URL + token, 
                  headers = {'Content-Type': 'application/x-www-form-urlencoded'}, auth=(username,password))

accesstoken = r.json()["accessToken"]

# Print the token
print("Access Token: " + accesstoken)

headers = {'Authorization':'Bearer ' + accesstoken}

### Print Available Collections

In [ ]:
# See what collections are available
r = requests.get(URL + collections, headers=headers)

# Print the results
#p(r.json())

### Post Search Filters, Print the Results

In [ ]:
# Post search filters
filters = {
  "bbox": [-122.66,37.58,-121.69,38.20], # lower left coodinate and upper right coordinate, in decimal degrees
  "datetime": "2017-01-01T00:00:00Z/2020-07-21T12:31:12Z", # start date/end date and time in YYYY-MM-DDTHH:MM:SSZ format
  "limit": 10, # overwrite the default pagination limit of 10, adjust as necessary
  "collections": ["sentinel-s1-l2"], # specify the desired collection  "sentinel-s1-l2"
  "sort": [{"field": "dtr:start_datetime"}],
}
headers = {'Content-Type': 'application/json',
  'Accept': 'application/geo+json', 'Authorization':'Bearer ' + accesstoken}
r = requests.post(URL + catsearch, json=filters, headers=headers)

# Inspect the results
p(r.json())

### Visualize Results on a Map

In [ ]:
# Display the results on a folium map
import folium

loc = [(filters["bbox"][1]+filters["bbox"][3])/2, (filters["bbox"][0]+filters["bbox"][2])/2] 
m = folium.Map(location=loc,zoom_start=5)
folium.GeoJson(
    r.text,
).add_to(m)

m

### Make and Post an Order

In [ ]:
# Make an Order
features = r.json()["features"]
granulelist = []

# Loop over all the features from the response and add to an array for an order
for f in features:
    item = {"CollectionId": f["collection"], "GranuleId": f["id"]}
    granulelist.append(item)
    
myorder = {"Items": granulelist}

# Post the order and inspect the result
r = requests.post(URL + orders, json=myorder, headers=headers)
p(r.json())

### Get the STAC records with the signed URLs using the /download endpoint, Print the Result

In [ ]:
myorderid = r.json()["orderId"]
r = requests.get(URL + orders + myorderid + '/download', headers=headers)
p(r.json())

### Download the Results

In [ ]:
features = r.json()
     
basefp = 'C:/data/S1/' # Local directory to save data

for feature in features:
           
    filepath = feature["assets"]["VH"]["href"] # the second nested dictionary ("VH" here) must be changed for different assets
    # e.g. filepath = feature["assets"]["VH-metadata"]["href"] will return the url for the assets .xml metadata file
    #print(filepath)

    filename = filepath[filepath.rfind("/")+1:]
    #print(filename)
    sep = "?"
    truncname = filename.split(sep, 1)[0]
    #print(truncname)
    
    outfp = basefp + truncname
    
    #import urllib
    #f = urllib.request.urlretrieve(filepath, outfp)
    with requests.get(filepath, stream=True) as result:
        result.raise_for_status()
        with open(outfp, 'wb') as f:
            for chunk in result.iter_content(chunk_size=10000000):
                f.write(chunk)      